In [2]:
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime
import os
import pandas as pd

def scraper_03(chrome_driver_path, data_website, year_to_select, cause_of_death):

    chrome_driver = webdriver.Chrome(chrome_driver_path)
    chrome_driver.get(data_website) # chrome_driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") 

    ################################################################################################################################
    # Criteria Filtering #
    ################################################################################################################################
    print(f"Start data scraping for year={year_to_select}, cause of death={cause_of_death}, by district x age...")

    # Year - selectively
    year_box = chrome_driver.find_element(By.XPATH, '//*[@id="cbo0"]')
    year_option_screening = year_box.find_elements(By.TAG_NAME, "option")
    year_box_option_list = [i.get_attribute("value") for i in year_option_screening]
    year_indexing = int(year_box_option_list.index(year_to_select))
    year_box_choose_one = year_option_screening[year_indexing]
    year_box_choose_one.click(); chrome_driver.implicitly_wait(5)
    
    # Gender=all, Age=all, District=all
    gender_box = chrome_driver.find_element(By.XPATH, '//*[@id="All1"]')
    age_box = chrome_driver.find_element(By.XPATH, '//*[@id="All2"]')
    district_box = chrome_driver.find_element(By.XPATH, '//*[@id="All3"]')
    gender_box.click(); age_box.click(); district_box.click()
    chrome_driver.implicitly_wait(5)

    # Cause of death - selectively
    death_cause_box = chrome_driver.find_element(By.XPATH, '//*[@id="cbo4"]')
    death_cause_option_screening = death_cause_box.find_elements(By.TAG_NAME, "option")
    death_cause_option_list = [i.get_attribute("value") for i in death_cause_option_screening]
    death_cause_indexing = int(death_cause_option_list.index(cause_of_death))
    death_box_choose_one = death_cause_option_screening[death_cause_indexing]
    death_box_choose_one.click()

    ################################################################################################################################
    # Requesting Table #
    ################################################################################################################################
    calculation_button = chrome_driver.find_element(By.XPATH, '//*[@id="bt1"]')
    calculation_button.click()
    chrome_driver.implicitly_wait(5)

    try: 
        pop_up = Alert(chrome_driver); pop_up_text = pop_up.text
        pop_up.accept()
        data_scraped = pd.DataFrame(
        index=['Central & Western', 'Eastern (HK)', 'Southern (HK)', 'Wan Chai','Kowloon City', 'Kwun Tong', 'Sham Shui Po', 'Wong Tai Sin',
                    'Yau Tsim Mong', 'Islands', 'Kwai Tsing', 'North', 'Sai Kung','Sha Tin', 'Tai Po', 'Tsuen Wan', 
                    'Tuen Mun', 'Yuen Long', 'Marine','Outside Hong Kong', 'Unknown', 'Total'],

        columns=[f"{cause_of_death}_0-4",f"{cause_of_death}_5-9", f"{cause_of_death}_10-14", f"{cause_of_death}_10-14",
                f"{cause_of_death}_15-19",f"{cause_of_death}_20-24", f"{cause_of_death}_25-29", f"{cause_of_death}_30-34",
                f"{cause_of_death}_35-39",f"{cause_of_death}_40-44", f"{cause_of_death}_45-49", f"{cause_of_death}_50-54",
                f"{cause_of_death}_55-59",f"{cause_of_death}_60-64", f"{cause_of_death}_65-69", f"{cause_of_death}_70-74",
                f"{cause_of_death}_75-79",f"{cause_of_death}_80-84", f"{cause_of_death}_85 and above", f"{cause_of_death}_Unknown",],
            
            ); data_scraped.fillna(pop_up_text, inplace=True)

        print(f"Done data scraping for year={year_to_select}, cause of death={cause_of_death}!")

    except:
        table_dropdown = chrome_driver.find_element(By.XPATH, '//*[@id="cboRowVar"]')
        table_dropdown_screening = table_dropdown.find_elements(By.TAG_NAME, "option")
        table_var_choose_one = table_dropdown_screening[2] ########################################################## 2=="Age Group"
        table_var_choose_one.click()

        show_table_button = chrome_driver.find_element(By.XPATH, '//*[@id="showhide"]/input')
        show_table_button.click()

        table = chrome_driver.find_element(By.XPATH, '//*[@id="bivContainer"]')
        index_screening = table.find_elements(By.TAG_NAME, "th")
        index_list = [i.text for i in index_screening]
        data_screening = table.find_elements(By.TAG_NAME, "td")
        data_list = [i.text for i in data_screening]

        reshaped_index = index_list[2:23]; reshaped_index.append("Total")
        reshaped_data = {
            f"{cause_of_death}_{index_list[24]}": data_list[1:1+22],
            f"{cause_of_death}_{index_list[25]}": data_list[23:23+22],
            f"{cause_of_death}_{index_list[26]}": data_list[45:45+22],
            f"{cause_of_death}_{index_list[27]}": data_list[67:67+22],
            f"{cause_of_death}_{index_list[28]}": data_list[89:89+22],
            f"{cause_of_death}_{index_list[29]}": data_list[111:111+22],
            f"{cause_of_death}_{index_list[30]}": data_list[133:133+22],
            f"{cause_of_death}_{index_list[31]}": data_list[155:155+22],
            f"{cause_of_death}_{index_list[32]}": data_list[177:177+22],
            f"{cause_of_death}_{index_list[33]}": data_list[199:199+22],
            f"{cause_of_death}_{index_list[34]}": data_list[221:221+22],
            f"{cause_of_death}_{index_list[35]}": data_list[243:243+22],
            f"{cause_of_death}_{index_list[36]}": data_list[265:265+22],
            f"{cause_of_death}_{index_list[37]}": data_list[287:287+22],
            f"{cause_of_death}_{index_list[38]}": data_list[309:309+22],
            f"{cause_of_death}_{index_list[39]}": data_list[331:331+22],
            f"{cause_of_death}_{index_list[40]}": data_list[353:353+22],
            f"{cause_of_death}_{index_list[41]}": data_list[375:375+22],
            f"{cause_of_death}_{index_list[42]}": data_list[397:397+22]
        }

        data_scraped = pd.DataFrame(
            index=reshaped_index,
            data=reshaped_data)
            
        new_col = [i.replace("years", "") for i in data_scraped.columns]
        data_scraped.columns = new_col

        print(f"Done data scraping for year={year_to_select}, cause of death={cause_of_death}!")
    
    chrome_driver.quit()

    return data_scraped.transpose()

def scraper_03_extended (local_driver_path, hksar_page, list_of_diseases, year):
    result_list=[]
    for i in list_of_diseases:
        interim_result = scraper_03(
            chrome_driver_path=local_driver_path,
            data_website=hksar_page,
            year_to_select=year,
            cause_of_death=i)
        
        result_list.append(interim_result)
        result_for_diseases=pd.concat(result_list)
        result_for_diseases=result_for_diseases.reset_index().rename(columns={"index":year})
    
    return result_for_diseases

################################################################################################################################
# Run #
################################################################################################################################
codebook = pd.read_excel("/Users/noel/Documents/GitHub/HKSAR_Mortality_Scraping/CodeBook_WebScraping_Mortality.xlsx")

df = scraper_03_extended(    
    local_driver_path="/Users/noel/Documents/GitHub/HKSAR_Mortality_Scraping/chromedriver_mac",
    hksar_page="https://www.healthyhk.gov.hk/phisweb/enquiry/mo_ysad10_indiv_e.html#resultanchor",
    year="2020", 
    list_of_diseases=["C33", "C34"]
); df 

/var/folders/ql/kkknqg550p134mxb24bj0clc0000gn/T/ipykernel_18208/1341216329.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  chrome_driver = webdriver.Chrome(chrome_driver_path)


Start data scraping for year=2020, cause of death=C33, by district x age...
Done data scraping for year=2020, cause of death=C33!


/var/folders/ql/kkknqg550p134mxb24bj0clc0000gn/T/ipykernel_18208/1341216329.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  chrome_driver = webdriver.Chrome(chrome_driver_path)


Start data scraping for year=2020, cause of death=C34, by district x age...
Done data scraping for year=2020, cause of death=C34!


,2020,Central & Western,Eastern (HK),Southern (HK),Wan Chai,Kowloon City,Kwun Tong,Sham Shui Po,Wong Tai Sin,Yau Tsim Mong,...,Sai Kung,Sha Tin,Tai Po,Tsuen Wan,Tuen Mun,Yuen Long,Marine,Outside Hong Kong,Unknown,Total
0,C33_0 - 4,---,---,---,---,---,---,---,---,---,...,---,---,---,---,---,---,---,---,---,---
1,C33_5 - 9,---,---,---,---,---,---,---,---,---,...,---,---,---,---,---,---,---,---,---,---
2,C33_10 - 14,---,---,---,---,---,---,---,---,---,...,---,---,---,---,---,---,---,---,---,---
3,C33_15 - 19,---,---,---,---,---,---,---,---,---,...,---,---,---,---,---,---,---,---,---,---
4,C33_20 - 24,---,---,---,---,---,---,---,---,---,...,---,---,---,---,---,---,---,---,---,---
5,C33_25 - 29,---,---,---,---,---,---,---,---,---,...,---,---,---,---,---,---,---,---,---,---
6,C33_30 - 34,---,---,---,---,---,---,---,---,---,...,---,---,---,---,---,---,---,---,---,---
7,C33_35 - 39,---,---,---,1,---,---,---,---,---,...,---,---,---,---,---,---,---,---,---,1
8,C33_40 - 44,---,---,---,---,---,---,---,---,---,...,---,---,---,---,---,---,---,---,---,---
9,C33_45 - 49,---,---,---,---,---,---,---,---,---,...,---,---,---,---,---,---,---,---,---,---
